# A/B Testing the Udacity Website

In these exercises, we'll be analyzing data on user behavior from an experiment run by Udacity, the online education company. More specifically, we'll be looking at a test Udacity ran to improve the onboarding process on their site.

Udacity's test is an example of an "A/B" test, in which some portion of users visiting a website (or using an app) are randomly selected to see a new version of the site. An analyst can then compare the behavior of users who see a new website design to users seeing their normal website to estimate the effect of rolling out the proposed changes to all users. While this kind of experiment has it's own name in industry (A/B testing), to be clear it's just a randomized experiment, and so everything we've learned about potential outcomes and randomized experiments apply here. 

(Udacity has generously provides the data from this test under an Apache open-source license, and you can find their [original writeup here](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook). If you're interested in learning more on A/B testing in particular, it seems only fair while we use their data to flag they have a full course on the subject [here](https://www.udacity.com/course/ab-testing--ud257).)

## Udacity's Test

The test [is described by Udacity as follows](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook): 

At the time of this experiment, Udacity courses currently have two options on the course overview page: "start free trial", and "access course materials".

**Current Conditions Before Change**

- If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first.
- If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback.

**Description of Experimented Change**

- In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course.
- If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.
- At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. This [screenshot](images/udacity_checkyoureready.png) shows what the experiment looks like.

**Udacity's Hope is that...**:

> this might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time -- without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/ids720_specific/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_abtesting.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}
```


### Submission Limits

Please remember that you are **only allowed THREE submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Import the Data

### Exercise 1

Begin by importing Udacity's data on user behavior [here.](https://github.com/nickeubank/MIDS_Data/tree/master/udacity_AB_testing) 

There are TWO datasets for this test — one for the control data (users who saw the original design), and one for treatment data (users who saw the experimental design). Udacity decided to show their test site to 1/2 of visitors, so there are roughly the same number of users appearing in each dataset (though this is not a requirement of AB tests).

Please remember to load the data directly from github to assist the autograder.

In [2]:
import pandas as pd

df_ctrl = pd.read_csv(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/udacity_AB_testing/control_data.csv"
)
df_exp = pd.read_csv(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/udacity_AB_testing/experiment_data.csv"
)

### Exercise 2

Explore the data. Can you identify the unit of observation of the data (e.g. what is represented by each row)?

To be clear, the columns represent stages in a user funnel:

- Some number of users arrive at the website and are counted as Pageviews,
- Some portion of those users then click to enroll (and are counted as clicks),
- Some portion of those users then actually enroll in the free trial (after seeing an informational popup, in the case of treatment individuals),
- Finally some portion of those users end up paying at the end of the free trial period.

(Note this is not the only way that A/B test data can be collected and/or reported — this is just what Udacity provided, presumably to help address privacy concerns.)

In [3]:
# df_ctrl.head(5)

In [4]:
# df_exp.head(5)

In [5]:
# print(df_exp.shape)
# print(df_ctrl.shape)

> Each row represents a day - and the columns represent the total count at each stage of the funnel (for the day respectively).

## Pick your measures

### Exercise 3

The easiest way to analyze this data is to stack it into a single dataset where each observation is a day-treatment-arm (so you should end up with two rows per day, one for those who are in the treated groups, and one for those who were in the control group). Note that currently nothing in the data identifies whether a given observation is a treatment group observation or a control group observation, so you'll want to make sure to add a "treatment" indicator variable.

The variables in the data are:

- Pageviews: number of unique users visiting homepage
- Clicks: number of those users clicking "Start Free Trial"
- Enrollments: Number of people enrolling in trial
- Payments: Number of people who eventually pay for the service. Note the `payment` column reports payments for the users who first visited the site on the reported date, not payments occurring on the reported date.

In [6]:
# Add a column, set to 1 if treated, 0 otherwise
df_ctrl["treated"] = 0
df_exp["treated"] = 1
df_both = pd.concat([df_ctrl, df_exp], axis=0, ignore_index=True)

In [7]:
#  df_ctrl.head(5)

In [8]:
# df_both.head(5)
# print(df_both.shape)

### Exercise 4

Given Udacity's goals, what outcome are they hoping will be impacted by their manipulation?

Or, to ask the same question in the language of the Potential Outcomes Framework, what is their $Y$?

Or to ask the same question in the language of Kohavi, Tang and Xu, what is their *Overall Evaluation Criterion (OEC)*?

(I'm only asking one question, I'm just trying to phrase it using different terminologies we've encountered to help you see how they all fit together)

When you feel like you have your answer, please compute it. Store the average value of the variable in `results` under the key `ex4_avg_oec`. **Please round your answer to 4 decimal places.**

NOTE: You'll probably notice you have two choices to make when it comes to actually computing the OEC. 

- You could probably imagine either computing a ratio or a difference of two things — please calculate the difference.
- You may also be unsure whether to normalize by `Clicks`. Normalizing by clicks will help account for variation that comes from day-to-day variation in users, so it's a good thing to do. With infinite data, you'd expect to get the same results without normalizing by `Clicks` (since on average the same share of users are in each arm of the experiment), but for finite data it's a good strategy. Note that this is only ok because users make the choice to click or not *before* they see different versions of the website (it is "pre-treatment").

Just to make sure you're on track, your measure should have an average value of *about* 9%.

In [9]:
df_both["diff"] = df_both["Enrollments"] - df_both["Payments"]
df_both["click_diff_ratio"] = df_both["diff"] / df_both["Clicks"]

# df_both.head(5)

In [10]:
# print(df_both["click_pay_ratio"].mean())

ex4_avg_oec = df_both["click_diff_ratio"].mean()
ex4_avg_oec = round(ex4_avg_oec, 4)
print(ex4_avg_oec)

0.0941


> Our measure is 'click_diff_ratio' which measures the proportion of users who continue their free-trial (difference between enrollment and payment) based on everyone who clicks

### Exercise 5

Given Udacity's goals, what outcome are they hoping will *not* be impacted by their manipulation? In other words, what do they want to measure to ensure their treatment doesn't have unintended negative consequences that might be really costly to their operation?

Note that while this isn't how Kohavi, Tang, and Xu use the term "guardrail metrics" — they usually use the term to refer to things we measure to ensure the experiment is working the way it should — some people would also use the term "guardrail metrics" for something that could be impacted even if the experiment is working correctly, but which the organization wants to track to ensure they aren't impacted because they are deemed really important.

Again, please normalize by `Clicks`. Store the average value of this guardrail metric as `ex5_avg_guardrail` and **round your answer to 4 decimal places.**

In [11]:
df_both.head()

,Date,Pageviews,Clicks,Enrollments,Payments,treated,diff,click_diff_ratio
0,"Sat, Oct 11",7723,687,134.0,70.0,0,64.0,0.093159
1,"Sun, Oct 12",9102,779,147.0,70.0,0,77.0,0.098845
2,"Mon, Oct 13",10511,909,167.0,95.0,0,72.0,0.079208
3,"Tue, Oct 14",9871,836,156.0,105.0,0,51.0,0.061005
4,"Wed, Oct 15",10014,837,163.0,64.0,0,99.0,0.118280


In [12]:
df_both["click_pay_ratio"] = df_both["Payments"] / df_both["Clicks"]
ex5_avg_guardrail = round(df_both["click_pay_ratio"].mean(), 4)
print(ex5_avg_guardrail)

0.1158


> Guardrail metric is the number of people who end up converting as a proportion of total clicks, as this is the bottomline of the business. They want to ensure the additional friction point doesn't eventually dissuade people who would've otherwise enrolled and paid despite not being able to dedicate enough time doesn't significantly affect their bottomline.

## Validating The Data

### Exercise 6

Whenever you are working with experimental data, the first thing you want to do is verify that users actually were randomly sorted into the two arms of the experiment. In this data, half of users were supposed to be shown the old version of the site and half were supposed to see the new version.

`Pageviews` tells you how many unique users visited the welcome site we are experimenting on. `Pageviews` is what is sometimes called an "invariant" or "guardrail" variable, meaning that it shouldn't vary across treatment arms—after all, people have to visit the site before they get a chance to see the treatment, so there's no way that being assigned to treatment or control should affect the number of pageviews assigned to each group.

"Invariant" variables are also an example of what are known as a "pre-treatment" variable, because pageviews are determined before users are manipulated in any way. That makes it analogous to gender or age in experiments where you have demographic data—a person's age and gender are determined before they experience any manipulations, so the value of any pre-treatment attributes should be the same across the two arms of our experiment. This is what we've previously called "checking for balance," If pre-treatment attributes aren't balanced, then we may worry our attempt to randomly assign people to different groups failed.  Kohavi, Tang and Xu call this a "trust-based guardrail metric" because it helps us determine if we should trust our data.

To test the quality of the randomization, calculate the average number of pageviews for the treated group and for the control group. Do they look similar?


In [13]:
pageview_control = round(df_both[df_both["treated"] == 0]["Pageviews"].mean(), 2)
pageview_treated = round(df_both[df_both["treated"] == 1]["Pageviews"].mean(), 2)
print("Control: " + str(pageview_control))
print("Treated: " + str(pageview_treated))
print("Difference: " + str(round(pageview_control - pageview_treated, 2)))

Control: 9339.0
Treated: 9315.14
Difference: 23.86


> They are fairly similar with a difference of 24 views (23.86 exactly) between the control and treated group, with control group having more overall page views.

### Exercise 7

"Similar" is a tricky concept -- obviously, we expect *some* differences across groups since users were *randomly* divided across treatment arms. The question is whether the differences between groups are larger than we'd expect to emerge given our random assignment process. To evaluate this, let's use a `ttest` to test the statistical significance of the differences we see. 

**Note**: Remember that scipy functions don't accept `pandas` objects, so you use a scipy function, you have to pass the numpy vectors underlying your data with the `.values` operator (e.g. `df.my_column.values`). 

Does the difference in `pageviews` look statistically significant?

Store the resulting p-value in `ex7_ttest_pvalue` **rounded to four decimal places.**

In [14]:
from scipy import stats

control_pageviews = df_both[df_both["treated"] == 0]["Pageviews"].values
treated_pageviews = df_both[df_both["treated"] == 1]["Pageviews"].values

t_stat, p_value = stats.ttest_ind(control_pageviews, treated_pageviews)

ex7_ttest_pvalue = round(p_value, 4)
print(ex7_ttest_pvalue)

0.8877


> The resulting p-value of 0.8877 is well above the 0.05 threshold, indicating that the difference in pageviews is not statistically significant. This confirms that the randomization process was successful and the groups are well-balanced.

### Exercise 8

`Pageviews` is not the only "pre-treatment" variable in this data we can use to evaluate balance/use as a guardrail metric. What other measure is pre-treatment? Review the description of the experiment if you're not sure.

> Clicks is the other pre-treatment variable. Since the experimental prompt is only shown after a user clicks "Start free trial," the behavior of clicking itself occurs before any manipulation takes place. Like pageviews, it serves as a "trust-based guardrail metric" to ensure the groups were balanced at the start of the trial.

### Exercise 9

Check if the other pre-treatment variable is also balanced. Store the p-value of your test of difference in `results` under the key `"ex9_ttest_pvalue_clicks"` **rounded to four decimal places.**

In [15]:
from scipy import stats

control_clicks = df_both[df_both["treated"] == 0]["Clicks"].values
treated_clicks = df_both[df_both["treated"] == 1]["Clicks"].values

t_stat, p_value = stats.ttest_ind(control_clicks, treated_clicks)

ex9_ttest_pvalue_clicks = round(p_value, 4)
print(ex9_ttest_pvalue_clicks)

0.9264


> The resulting p-value of 0.9264 is well above the 0.05 threshold, indicating that the difference in clicks is not statistically significant.

## Estimating the Effect of Experiment

### Exercise 10

Now that we've validated our randomization, our next task is to estimate our treatment effect. First, though, there's an issue with your data you've been able to largely ignore until now, but which you should get a grip on before estimating your treatment effect — can you tell what it is and what you should do about it?

Store the number of observations in your data *after* you've addressed this in `ex10_num_obs` (this is mostly meant as a way to sanity check your answer with autograder).

In [16]:
print(df_both.isnull().sum())

Date                 0
Pageviews            0
Clicks               0
Enrollments         28
Payments            28
treated              0
diff                28
click_diff_ratio    28
click_pay_ratio     28
dtype: int64


In [17]:
print(df_both[df_both.isnull().any(axis=1)])

           Date  Pageviews  Clicks  Enrollments  Payments  treated  diff  \
23   Mon, Nov 3       9437     788          NaN       NaN        0   NaN   
24   Tue, Nov 4       9420     781          NaN       NaN        0   NaN   
25   Wed, Nov 5       9570     805          NaN       NaN        0   NaN   
26   Thu, Nov 6       9921     830          NaN       NaN        0   NaN   
27   Fri, Nov 7       9424     781          NaN       NaN        0   NaN   
28   Sat, Nov 8       9010     756          NaN       NaN        0   NaN   
29   Sun, Nov 9       9656     825          NaN       NaN        0   NaN   
30  Mon, Nov 10      10419     874          NaN       NaN        0   NaN   
31  Tue, Nov 11       9880     830          NaN       NaN        0   NaN   
32  Wed, Nov 12      10134     801          NaN       NaN        0   NaN   
33  Thu, Nov 13       9717     814          NaN       NaN        0   NaN   
34  Fri, Nov 14       9192     735          NaN       NaN        0   NaN   
35  Sat, Nov

In [18]:
df_cleaned = df_both.dropna(subset=["Enrollments", "Payments"])

ex10_num_obs = len(df_cleaned)
print(ex10_num_obs)

46



### Exercise 11

Now that we've established we have good balance (meaning we think randomization was likely successful), we can evaluate the effects of the experiment. Test whether the OEC and the metric you *don't* want affected have different average values in the control group and treatment group. 

Because we've randomized, this is a consistent estimate of the Average Treatment Effect of Udacity's website change.

Calculate the difference in means in your OEC and guardrail metrics using a simple t-test. Store the resulting effect estimates in `ex11_oec_ate` and `ex11_guard_ate` and p-values in `ex11_oec_pvalue` and `ex11_guard_pvalue`. **Please round all answers to 4 decimal places.** Report your ATE in *percentage points*, where `1` denotes 1 percentage point.


In [ ]:
control = df_cleaned[df_cleaned["treated"] == 0]
treated = df_cleaned[df_cleaned["treated"] == 1]

oec_diff = (
    treated["click_diff_ratio"].mean() - control["click_diff_ratio"].mean()
) * 100
ex11_oec_ate = round(oec_diff, 4)

t_stat_oec, p_val_oec = stats.ttest_ind(
    treated["click_diff_ratio"], control["click_diff_ratio"]
)
ex11_oec_pvalue = round(p_val_oec, 4)

# 2. Guardrail
guard_diff = (
    treated["click_pay_ratio"].mean() - control["click_pay_ratio"].mean()
) * 100
ex11_guard_ate = round(guard_diff, 4)

# Guardrail P-value
t_stat_guard, p_val_guard = stats.ttest_ind(
    treated["click_pay_ratio"], control["click_pay_ratio"]
)
ex11_guard_pvalue = round(p_val_guard, 4)

# Store results
print("ex11_oec_ate:", +ex11_oec_ate)
print("ex11_oec_pvalue:", +ex11_oec_pvalue)
print("ex11_guard_ate:", +ex11_guard_ate)
print("ex11_guard_pvalue:", +ex11_guard_pvalue)

ex11_oec_ate: -1.5888
ex11_oec_pvalue: 0.1319
ex11_guard_ate: -0.4897
ex11_guard_pvalue: 0.5928


### Exercise 12

Do you feel that Udacity achieved their goal? Did their intervention cause them any problems? If they asked you "What would happen if we rolled this out to everyone?" what would you say?

As you answer this question, a small additional question: up until this point you've (presumably) been reporting the default p-values from the tools you are using. These, as you may recall from stats 101, are two-tailed p-values. Do those seem appropriate for your OEC?

> Did Udacity achieve their goal? No - while the treatment reduced the enrollment-payment gap by 1.59 percentage points in the desired direction, this effect is not statistically significant (p=0.13).

> Did their intervention cause problems? The guardrail metric (payment rate) decreased slightly by 0.49 percentage points, but this is also not statistically significant (p=0.59).

> What would happen if rolled out? I would not recommend rollout - there's no significant evidence of benefit, and the point estimates suggest potential harm to payment rates without clear gains.

> Two-tailed vs. One-tailed: A one-tailed test is more appropriate here since Udacity had a specific directional hypothesis that the intervention would decrease dropouts. However, even with a one-tailed test, the p-value (≈0.066) still fails to meet the standard 0.05 significance threshold.

### Exercise 13

One of the magic things about experiments is that all you have to do is compare averages to get an average treatment effect. However, you *can* do other things to try and increase the statistical power of your experiments, like add controls in a linear regression model. 

As you likely know, a bivariate regression is exactly equivalent to a t-test, so let's start by re-estimating the effect of treatment on your OEC using a linear regression. Can you replicate the results from your t-test? They shouldn't just be close—they should be numerically equivalent (i.e. exactly the same to the limits of floating point number precision). 

In [23]:
import statsmodels.formula.api as smf

ols_model = smf.ols("click_diff_ratio ~ treated", data=df_cleaned).fit()

ate_regression = round(ols_model.params["treated"] * 100, 4)
p_value_regression = round(ols_model.pvalues["treated"], 4)
print("Regression ATE: " + str(ate_regression))
print("Regression P-value: " + str(p_value_regression))

Regression ATE: -1.5888
Regression P-value: 0.1319


> The values are indeed identical.

### Exercise 14

Now add indicator variables for the date of each observation. Do the standard errors on your `treatment` variable change? If so, in what direction?

Store your new standard error in `ex14_se_treatment`. Round your answer to 4 decimal places.

In [ ]:
res_date_controls = smf.ols(
    "click_diff_ratio ~ treated + C(Date)", data=df_cleaned
).fit()
ex14_se_treatment = round(res_date_controls.bse["treated"], 4)

print("New Standard Error:" + str(ex14_se_treatment))

se_base = ols_model.bse["treated"]
se_controlled = res_date_controls.bse["treated"]
percent_reduction = ((se_base - se_controlled) / se_base) * 100

print("Standard Error Reduction:" + str(percent_reduction))

New Standard Error:0.0066
Standard Error Reduction:35.987286921034915


You should have found that your standard errors decreased by about 30\%—this is why, although just comparing means *works*, if you have additional variables adding them to your analysis can be helpful (all the usual rules for model specification apply — for example, you still want to be careful about overfitting, which one could argue is maybe part of what's happening here). 

In many other cases, the effect of adding controls is likely to be larger — the date indicators we added to our data are perfectly balanced between treatment and control, so we aren't adding a lot of data to the model by adding them as variables. They're accounting for some day-to-day variation (presumably in the types of people coming to the site), but they aren't controlling for any residual baseline differences the way a control like "gender" or "age" might (since those kind of individual-level attributes will never be perfectly balanced across treatment and control). 

### Exercise 15

Does this result have any impact on the recommendations you would offer Udacity?

> While the added controls reduced the standard error and improved precision, the p-value remains above 0.05, meaning the result is still not statistically significant. This confirms that the intervention's effect is too small or inconsistent to justify a full rollout based on the current data.